In [1]:
!pip install backtrader

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats as st

import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers

import math
import datetime
import pytz

# from ipynb.fs.full.Task1 import get_files
# from ipynb.fs.full.Task2 import find_suitable_pairs
from BackTrader import *
# from BackTrader import PandasData, PairTradingStrategy
# from BackTrader import CommInfoFloat, DataAnalyzer, CashValueAnalyzer, OrderAnalyzer

In [3]:
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers
from enum import Enum
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib

import math
import datetime

#This is the data feed to be fed into the backtesting lib
class PandasData(btfeeds.PandasData):
    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Adj Close'),
        ('openinterest', None),
    )
    
class Log(btind.Indicator):
    """Calculates log."""
    lines = ('log',)
    
    def next(self):
        self.l.log[0] = math.log(self.data[0])

class OLSSlopeIntercept(btind.PeriodN):
    """Calculates a linear regression using OLS."""
    _mindatas = 2  # ensure at least 2 data feeds are passed

    lines = ('slope', 'intercept',)
    params = (
        ('period', 10),
    )

    def next(self):
        p0 = pd.Series(self.data0.get(size=self.p.period))
        p1 = pd.Series(self.data1.get(size=self.p.period))
        p1 = sm.add_constant(p1)
        intercept, slope = sm.OLS(p0, p1).fit().params

        self.lines.slope[0] = slope
        self.lines.intercept[0] = intercept

class OLSSpread(btind.PeriodN):
    """Calculates the z-score of the OLS spread."""
    _mindatas = 2  # ensure at least 2 data feeds are passed
    lines = ('slope', 'spread', 'spread_mean', 'spread_std', 'zscore',)
    params = (('period', 10),)

    def __init__(self):
        data0_log = Log(self.data0)
        data1_log = Log(self.data1)
        slint = OLSSlopeIntercept(data0_log, data1_log, period=self.p.period)

        spread = data0_log - (slint.slope * data1_log + slint.intercept)
        self.l.spread = spread
        self.l.slope = slint.slope

        self.l.spread_mean = bt.ind.SMA(spread, period=self.p.period)
        self.l.spread_std = bt.ind.StdDev(spread, period=self.p.period)
        self.l.zscore = (spread - self.l.spread_mean) / self.l.spread_std
        
class Status(Enum):
    LONG = 1
    SHORT = 2
    NONE = 3
    
class PairTradingStrategy(bt.Strategy):
    """Basic pair trading strategy."""
    
    # These are just default values, pass in these values while initialising strategy
    params = dict(
        period=100,
        order_pct1=0.1,
        order_pct2=0.1,
        printout=True,
        upper=2,
        lower=2,
        symbol1="SYMBOL1",
        symbol2="SYMBOL2"
    )

    def log(self, txt, dt=None):
        if self.p.printout:
            dt = dt or self.data.datetime[0]
            dt = bt.num2date(dt)
            print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
            return  # Await further notifications

        if order.status == order.Completed:
            if order.isbuy():
                buytxt = 'BUY COMPLETE {}, size = {:.2f}, price = {:.2f}'.format(
                    order.data._name, order.executed.size, order.executed.price)
                self.log(buytxt, order.executed.dt)
            else:
                selltxt = 'SELL COMPLETE {}, size = {:.2f}, price = {:.2f}'.format(
                    order.data._name, order.executed.size, order.executed.price)
                self.log(selltxt, order.executed.dt)

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            self.log('%s ,' % order.Status[order.status])
            pass  # Simply log

        # Allow new orders
        self.orderid = None

    def __init__(self):
        # To control operation entries
        self.orderid = None
        self.order_pct1 = self.p.order_pct1
        self.order_pct2 = self.p.order_pct2
        self.upper = self.p.upper
        self.lower = self.p.lower
        
        self.transform = OLSSpread(self.data0, self.data1, period=self.p.period)

        self.spread = self.transform.spread
        self.zscore = self.transform.zscore
        self.slope = self.transform.slope
        
        self.status = Status.NONE
        self.symbol1 = self.p.symbol1
        self.symbol2 = self.p.symbol2

    def next(self):
        if self.orderid:
            return  # if an order is active, no new orders are allowed
        
        if self.zscore[0] > self.upper and self.status != Status.SHORT:
            # Short sell stock 1
            self.order_target_percent(data=self.data0, target=-self.order_pct1)
            # Buy stock 2
            self.order_target_percent(data=self.data1, target=self.order_pct2)
            
            self.status = Status.SHORT

        elif self.zscore[0] < self.lower and self.status != Status.LONG:
            # Short sell stock 2
            self.order_target_percent(data=self.data1, target=-self.order_pct2)
            # Buy stock 1
            self.order_target_percent(data=self.data0, target=self.order_pct1)
                     
            self.status = Status.LONG
        
#         # Sample strategy does not close position when zscore reaches zero again, 
#         # only reverses position when it swings the other way
#         # uncomment below if desired behaviour is to close at z-score = 0
#         elif self.zscore[0] <= 0 and self.status == Status.SHORT:
#             # Close position
#             self.order_target_percent(data=self.data1, target=0)
#             self.order_target_percent(data=self.data0, target=0)
#             self.status = Status.NONE
        
#         elif self.zscore[0] >= 0 and self.status == Status.LONG:
#             # Close position
#             self.order_target_percent(data=self.data1, target=0)
#             self.order_target_percent(data=self.data0, target=0)
#             self.status = Status.NONE

    def stop(self):
        if self.p.printout:
            print('==================================================')
            print('Starting Value - %.2f' % self.broker.startingcash)
            print('Ending   Value - %.2f' % self.broker.getvalue())
            print('==================================================')


# This this can be added to Cerebro allow fractional shares (for this like crypto)
# By default backtrader doesnt allow that.
class CommInfoFloat(bt.CommInfoBase):
    """Commission schema that keeps size as float."""
    params = (
        ('stocklike', True),
        ('commtype', bt.CommInfoBase.COMM_PERC),
        ('percabs', True),
      )
    
    def getsize(self, price, cash):
        if not self._stocklike:
            return self.p.leverage * (cash / self.get_margin(price))

        return self.p.leverage * (cash / price)


class DataAnalyzer(bt.analyzers.Analyzer):
    """Analyzer to extract OHLCV."""
    def create_analysis(self):
        self.rets0 = {}
        self.rets1 = {}

    def next(self):
        self.rets0[self.strategy.datetime.datetime()] = [
            self.data0.open[0],
            self.data0.high[0],
            self.data0.low[0],
            self.data0.close[0],
            self.data0.volume[0]
        ]
        self.rets1[self.strategy.datetime.datetime()] = [
            self.data1.open[0],
            self.data1.high[0],
            self.data1.low[0],
            self.data1.close[0],
            self.data1.volume[0]
        ]

    def get_analysis(self):
        return self.rets0, self.rets1

class CashValueAnalyzer(bt.analyzers.Analyzer):
    """Analyzer to extract cash and value."""
    def create_analysis(self):
        self.rets = {}

    def notify_cashvalue(self, cash, value):
        self.rets[self.strategy.datetime.datetime()] = self.strategy.broker.getvalue()

    def get_analysis(self):
        return self.rets
    
class OrderAnalyzer(bt.analyzers.Analyzer):
    """Analyzer to extract order price, size, value, and paid commission."""
    def create_analysis(self):
        self.rets0 = {}
        self.rets1 = {}

    def notify_order(self, order):
        if order.status == order.Completed:
            if order.data._name == self.strategy.symbol1:
                rets = self.rets0
            else:
                rets = self.rets1
            rets[self.strategy.datetime.datetime()] = (
                order.executed.price,
                order.executed.size,
                -order.executed.size * order.executed.price,
                order.executed.comm
            )

    def get_analysis(self):
        return self.rets0, self.rets1

In [4]:
def calculate_volatility(values):
    values = pd.Series(values)
    log_return = np.log(values/values.shift())
    volatility = log_return.std()*252**.5
    return volatility*100

In [5]:
start_date = '2015-01-01'
end_date = '2023-01-01'

In [6]:
import yfinance as yf
SP500 = yf.download("^GSPC", start=start_date, end=end_date)
SP500 = SP500.drop(columns=['Volume'])
SP500

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close
Date,,,,,
2015-01-02,2058.899902,2072.360107,2046.040039,2058.199951,2058.199951
2015-01-05,2054.439941,2054.439941,2017.339966,2020.579956,2020.579956
2015-01-06,2022.150024,2030.250000,1992.439941,2002.609985,2002.609985
2015-01-07,2005.550049,2029.609985,2005.550049,2025.900024,2025.900024
2015-01-08,2030.609985,2064.080078,2030.609985,2062.139893,2062.139893
...,...,...,...,...,...
2022-12-23,3815.110107,3845.800049,3797.010010,3844.820068,3844.820068
2022-12-27,3843.340088,3846.649902,3813.219971,3829.250000,3829.250000
2022-12-28,3829.560059,3848.320068,3780.780029,3783.219971,3783.219971


In [7]:
class BuyAndHold(bt.Strategy):
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        self.order_target_value(target=self.broker.get_cash())

    def stop(self):
        pass
#         # calculate the actual returns
#         self.roi = (self.broker.get_value() / self.val_start) - 1.0
#         print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [8]:
cerebro = bt.Cerebro()
sp500_datafeed = PandasData(dataname=SP500, name="SP500")
cerebro.adddata(sp500_datafeed)
cerebro.addstrategy(BuyAndHold)
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(btanalyzers.SharpeRatio_A, _name='sharpe')
cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
cerebro.broker.setcash(100000)
res = cerebro.run()[0]
print("Sharpe Ratio: ", res.analyzers.sharpe.get_analysis()["sharperatio"])
print("Annualised Return: ", res.analyzers.returns.get_analysis()["rnorm100"])
print("Max Drawdown: ", res.analyzers.drawdown.get_analysis()["max"]["drawdown"])
print("Annualised Volatilty: ", calculate_volatility(SP500["Close"]))
# print()
# pairs.at[indexes[index], "SharpeRatio"] = res.analyzers.sharpe.get_analysis()["sharperatio"]
#         pairs.at[indexes[index], "Annualised Return(%)"] = res.analyzers.returns.get_analysis()["rnorm100"]
#         pairs.at[indexes[index], "Max Drawdown(%)"] = res.analyzers.drawdown.get_analysis()["max"]["drawdown"]
#         pairs.at[indexes[index], "Annualised Volatility(%)"] = volatility

Sharpe Ratio:  0.5249251819112056
Annualised Return:  8.05126276026581
Max Drawdown:  33.63793372479855
Annualised Volatilty:  18.93592092142449


In [9]:
PRICE_DIR = "individual_data"
STOCK_PRICE = {}

In [10]:
# Adjust these params for backtesting
PARAMS = {
    "CASH" : 100000,
    "UPPER" : st.norm.ppf(1 - 0.05 / 2),
    "LOWER" : -st.norm.ppf(1 - 0.05 / 2),
    "ORDER_PCT1" : 0.5, # Essentially order size (how much pct of current portfolio you want to spend for stock)
    "ORDER_PCT2" : 0.5,
    "PERIOD" : 100, # Look back period for doing rolling OLS and z-score calc.
    "COMMPERC": 0.005 # 0.5%
}

def run_backtest(stock1_name, stock2_name, stock1_df, stock2_df, PARAMS):
    stock1_datafeed = PandasData(dataname=stock1_df, name=stock1_name)
    stock2_datafeed = PandasData(dataname=stock2_df, name=stock2_name)

    cerebro = bt.Cerebro()

    cerebro.adddata(stock1_datafeed)
    cerebro.adddata(stock2_datafeed)

    cerebro.addstrategy(PairTradingStrategy, 
                        period=PARAMS["PERIOD"], 
                        upper=PARAMS["UPPER"], 
                        lower=PARAMS["LOWER"], 
                        order_pct1=PARAMS["ORDER_PCT1"],
                        order_pct2=PARAMS["ORDER_PCT2"],
                        symbol1=stock1_name,
                        symbol2=stock2_name,
                        printout=False)

    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(btanalyzers.SharpeRatio_A, _name='sharpe')
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='ta')
    cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
#     cerebro.addanalyzer(btanalyzers.LogReturnsRolling, _name='logrollingreturns')
    cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annualreturn')
#     cerebro.addanalyzer(btanalyzers.PositionsValue, _name='positionvalues')
    
    # Below are custom analyzers defined by the sample code, can uncomment if you need more detailed info
#     cerebro.addanalyzer(DataAnalyzer)
    cerebro.addanalyzer(CashValueAnalyzer)
    cerebro.addanalyzer(OrderAnalyzer)

    cerebro.broker.setcash(PARAMS["CASH"])
    comminfo = CommInfoFloat(commission=PARAMS["COMMPERC"])
    cerebro.broker.addcommissioninfo(comminfo)

    
    backtest_res = cerebro.run()[0]
    return backtest_res

In [11]:
filtered_pairs = pd.read_pickle("Data/filtered_pairs_dict.pickle")

In [17]:
filtered_pairs

{'KMeans':                                                    pair        pvalue  \
 0        (comms-comms - AMX.csv, comms-comms - BCE.csv)  8.992544e-26   
 1       (comms-comms - AMX.csv, comms-comms - CHTR.csv)  1.607755e-23   
 2      (comms-comms - AMX.csv, comms-comms - CMCSA.csv)  1.447099e-19   
 3        (comms-comms - AMX.csv, comms-comms - DIS.csv)  4.989754e-25   
 4       (comms-comms - AMX.csv, comms-comms - RELX.csv)  2.478912e-24   
 ...                                                 ...           ...   
 4602  (realestate-realestate - ARE.csv, utilities-ut...  2.859040e-29   
 4606  (realestate-realestate - ARE.csv, utilities-ut...  4.190126e-15   
 4608  (realestate-realestate - ARE.csv, utilities-ut...  1.555391e-29   
 4609  (realestate-realestate - ARE.csv, utilities-ut...  0.000000e+00   
 4613  (realestate-realestate - ARE.csv, utilities-ut...  2.256789e-29   
 
       hurst_exp   half_life  avg_cross_count  SharpeRatio  \
 0     -0.000929   30.519410       128

In [12]:
start_test = '2021-01-01'
end_test = '2022-12-30'

In [13]:
TESTING_START = datetime.datetime(2021, 1, 1)
TESTING_END = datetime.datetime(2022, 12, 30)
backtest_results_raw = {}

In [14]:
def read_stock_price(stock_name):
    stock_name = stock_name.replace("-", "/", 1)
    if stock_name in STOCK_PRICE:
        print(STOCK_PRICE[stock_name])
        return STOCK_PRICE[stock_name]
    file_path = PRICE_DIR + "/" + stock_name
    df = pd.read_csv(file_path)
    df = df.drop(0)
    df = df.ffill()
    dates = pd.to_datetime(df["Date"], utc=True).dt.date
    dates = pd.to_datetime(dates)
    df.index = dates
    df = df.loc[df.index >= TESTING_START]
    df = df.loc[df.index <= TESTING_END]
    STOCK_PRICE[stock_name] = df
    return df

In [15]:
for pair_strat, pairs in filtered_pairs.items():
    print("Doing Backtesting for ", pair_strat)
    pairs["SharpeRatio"] = np.nan
    pairs["Annualised Return(%)"] = np.nan
    pairs["Max Drawdown(%)"] = np.nan
    pairs["Annualised Volatility(%)"] = np.nan
    raw_results = {}
    indexes = pairs.index
    for (index, (stock1, stock2)) in enumerate(pairs["pair"]):
        print("Backtesting for ", (stock1, stock2))
        stock1_df = read_stock_price(stock1)
        stock2_df = read_stock_price(stock2)
        if stock1_df.isnull().values.any() or stock2_df.isnull().values.any():
            print(f'{stock1} or {stock2} has null values')
            continue
        res = run_backtest(stock1, stock2, stock1_df, stock2_df, PARAMS)
        portfolio_value = res.analyzers.cashvalueanalyzer.get_analysis()
        volatility = calculate_volatility(portfolio_value.values())
        orders = res.analyzers.orderanalyzer.get_analysis()
        
        # Raw results store the portfolio value at each date, and also the specific orders which were made during backtesting
        raw_results[(stock1, stock2)] = (portfolio_value, orders)        
        
        pairs.at[indexes[index], "SharpeRatio"] = res.analyzers.sharpe.get_analysis()["sharperatio"]
        pairs.at[indexes[index], "Annualised Return(%)"] = res.analyzers.returns.get_analysis()["rnorm100"]
        pairs.at[indexes[index], "Max Drawdown(%)"] = res.analyzers.drawdown.get_analysis()["max"]["drawdown"]
        pairs.at[indexes[index], "Annualised Volatility(%)"] = volatility
    backtest_results_raw[pair_strat] = raw_results

        

Doing Backtesting for  KMeans
Backtesting for  ('comms-comms - AMX.csv', 'comms-comms - BCE.csv')
Backtesting for  ('comms-comms - AMX.csv', 'comms-comms - CHTR.csv')
            Unnamed: 0        Date       Open   High        Low      Close  \
Date                                                                         
2021-01-04        1511  2021-01-04  14.670000  14.95  14.670000  14.750000   
2021-01-05        1512  2021-01-05  14.780000  14.93  14.570000  14.880000   
2021-01-06        1513  2021-01-06  14.910000  15.24  14.910000  15.070000   
2021-01-07        1514  2021-01-07  15.120000  15.12  14.790000  14.850000   
2021-01-08        1515  2021-01-08  14.910000  15.11  14.880000  14.960000   
...                ...         ...        ...    ...        ...        ...   
2022-12-23        2009  2022-12-23  18.580000  18.90  18.440001  18.740000   
2022-12-27        2010  2022-12-27  18.709999  18.82  18.540001  18.639999   
2022-12-28        2011  2022-12-28  18.670000  18.75 

Backtesting for  ('comms-comms - AMX.csv', 'comms-comms - TMUS.csv')
            Unnamed: 0        Date       Open   High        Low      Close  \
Date                                                                         
2021-01-04        1511  2021-01-04  14.670000  14.95  14.670000  14.750000   
2021-01-05        1512  2021-01-05  14.780000  14.93  14.570000  14.880000   
2021-01-06        1513  2021-01-06  14.910000  15.24  14.910000  15.070000   
2021-01-07        1514  2021-01-07  15.120000  15.12  14.790000  14.850000   
2021-01-08        1515  2021-01-08  14.910000  15.11  14.880000  14.960000   
...                ...         ...        ...    ...        ...        ...   
2022-12-23        2009  2022-12-23  18.580000  18.90  18.440001  18.740000   
2022-12-27        2010  2022-12-27  18.709999  18.82  18.540001  18.639999   
2022-12-28        2011  2022-12-28  18.670000  18.75  18.290001  18.340000   
2022-12-29        2012  2022-12-29  18.440001  18.58  18.360001  18.43000

Backtesting for  ('comms-comms - AMX.csv', 'conscycl-conscycl - MCD.csv')
            Unnamed: 0        Date       Open   High        Low      Close  \
Date                                                                         
2021-01-04        1511  2021-01-04  14.670000  14.95  14.670000  14.750000   
2021-01-05        1512  2021-01-05  14.780000  14.93  14.570000  14.880000   
2021-01-06        1513  2021-01-06  14.910000  15.24  14.910000  15.070000   
2021-01-07        1514  2021-01-07  15.120000  15.12  14.790000  14.850000   
2021-01-08        1515  2021-01-08  14.910000  15.11  14.880000  14.960000   
...                ...         ...        ...    ...        ...        ...   
2022-12-23        2009  2022-12-23  18.580000  18.90  18.440001  18.740000   
2022-12-27        2010  2022-12-27  18.709999  18.82  18.540001  18.639999   
2022-12-28        2011  2022-12-28  18.670000  18.75  18.290001  18.340000   
2022-12-29        2012  2022-12-29  18.440001  18.58  18.360001  18.

Backtesting for  ('comms-comms - AMX.csv', 'consdef-consdef - ADM.csv')
            Unnamed: 0        Date       Open   High        Low      Close  \
Date                                                                         
2021-01-04        1511  2021-01-04  14.670000  14.95  14.670000  14.750000   
2021-01-05        1512  2021-01-05  14.780000  14.93  14.570000  14.880000   
2021-01-06        1513  2021-01-06  14.910000  15.24  14.910000  15.070000   
2021-01-07        1514  2021-01-07  15.120000  15.12  14.790000  14.850000   
2021-01-08        1515  2021-01-08  14.910000  15.11  14.880000  14.960000   
...                ...         ...        ...    ...        ...        ...   
2022-12-23        2009  2022-12-23  18.580000  18.90  18.440001  18.740000   
2022-12-27        2010  2022-12-27  18.709999  18.82  18.540001  18.639999   
2022-12-28        2011  2022-12-28  18.670000  18.75  18.290001  18.340000   
2022-12-29        2012  2022-12-29  18.440001  18.58  18.360001  18.43

Backtesting for  ('comms-comms - AMX.csv', 'consdef-consdef - DEO.csv')
            Unnamed: 0        Date       Open   High        Low      Close  \
Date                                                                         
2021-01-04        1511  2021-01-04  14.670000  14.95  14.670000  14.750000   
2021-01-05        1512  2021-01-05  14.780000  14.93  14.570000  14.880000   
2021-01-06        1513  2021-01-06  14.910000  15.24  14.910000  15.070000   
2021-01-07        1514  2021-01-07  15.120000  15.12  14.790000  14.850000   
2021-01-08        1515  2021-01-08  14.910000  15.11  14.880000  14.960000   
...                ...         ...        ...    ...        ...        ...   
2022-12-23        2009  2022-12-23  18.580000  18.90  18.440001  18.740000   
2022-12-27        2010  2022-12-27  18.709999  18.82  18.540001  18.639999   
2022-12-28        2011  2022-12-28  18.670000  18.75  18.290001  18.340000   
2022-12-29        2012  2022-12-29  18.440001  18.58  18.360001  18.43

...

Backtesting for  ('comms-comms - ATVI.csv', 'utilities-utilities - D.csv')
            Unnamed: 0        Date       Open       High        Low  \
Date                                                                  
2021-01-04        1511  2021-01-04  92.699997  92.910004  89.220001   
2021-01-05        1512  2021-01-05  90.320000  90.750000  89.099998   
2021-01-06        1513  2021-01-06  89.059998  89.900002  87.889999   
2021-01-07        1514  2021-01-07  88.860001  90.750000  88.269997   
2021-01-08        1515  2021-01-08  90.070000  91.370003  89.529999   
...                ...         ...        ...        ...        ...   
2022-12-23        2009  2022-12-23  75.699997  76.070000  75.500000   
2022-12-27        2010  2022-12-27  75.900002  76.050003  75.540001   
2022-12-28        2011  2022-12-28  75.639999  76.480003  75.500000   
2022-12-29        2012  2022-12-29  76.040001  76.870003  76.029999   
2022-12-30        2013  2022-12-30  76.500000  76.660004  76.220001   

 

Backtesting for  ('comms-comms - ATVI.csv', 'utilities-utilities - EIX.csv')
            Unnamed: 0        Date       Open       High        Low  \
Date                                                                  
2021-01-04        1511  2021-01-04  92.699997  92.910004  89.220001   
2021-01-05        1512  2021-01-05  90.320000  90.750000  89.099998   
2021-01-06        1513  2021-01-06  89.059998  89.900002  87.889999   
2021-01-07        1514  2021-01-07  88.860001  90.750000  88.269997   
2021-01-08        1515  2021-01-08  90.070000  91.370003  89.529999   
...                ...         ...        ...        ...        ...   
2022-12-23        2009  2022-12-23  75.699997  76.070000  75.500000   
2022-12-27        2010  2022-12-27  75.900002  76.050003  75.540001   
2022-12-28        2011  2022-12-28  75.639999  76.480003  75.500000   
2022-12-29        2012  2022-12-29  76.040001  76.870003  76.029999   
2022-12-30        2013  2022-12-30  76.500000  76.660004  76.220001   


Backtesting for  ('comms-comms - ATVI.csv', 'utilities-utilities - FE.csv')
            Unnamed: 0        Date       Open       High        Low  \
Date                                                                  
2021-01-04        1511  2021-01-04  92.699997  92.910004  89.220001   
2021-01-05        1512  2021-01-05  90.320000  90.750000  89.099998   
2021-01-06        1513  2021-01-06  89.059998  89.900002  87.889999   
2021-01-07        1514  2021-01-07  88.860001  90.750000  88.269997   
2021-01-08        1515  2021-01-08  90.070000  91.370003  89.529999   
...                ...         ...        ...        ...        ...   
2022-12-23        2009  2022-12-23  75.699997  76.070000  75.500000   
2022-12-27        2010  2022-12-27  75.900002  76.050003  75.540001   
2022-12-28        2011  2022-12-28  75.639999  76.480003  75.500000   
2022-12-29        2012  2022-12-29  76.040001  76.870003  76.029999   
2022-12-30        2013  2022-12-30  76.500000  76.660004  76.220001   



Backtesting for  ('comms-comms - ATVI.csv', 'utilities-utilities - NGG.csv')
            Unnamed: 0        Date       Open       High        Low  \
Date                                                                  
2021-01-04        1511  2021-01-04  92.699997  92.910004  89.220001   
2021-01-05        1512  2021-01-05  90.320000  90.750000  89.099998   
2021-01-06        1513  2021-01-06  89.059998  89.900002  87.889999   
2021-01-07        1514  2021-01-07  88.860001  90.750000  88.269997   
2021-01-08        1515  2021-01-08  90.070000  91.370003  89.529999   
...                ...         ...        ...        ...        ...   
2022-12-23        2009  2022-12-23  75.699997  76.070000  75.500000   
2022-12-27        2010  2022-12-27  75.900002  76.050003  75.540001   
2022-12-28        2011  2022-12-28  75.639999  76.480003  75.500000   
2022-12-29        2012  2022-12-29  76.040001  76.870003  76.029999   
2022-12-30        2013  2022-12-30  76.500000  76.660004  76.220001   


Backtesting for  ('comms-comms - ATVI.csv', 'utilities-utilities - SRE.csv')
            Unnamed: 0        Date       Open       High        Low  \
Date                                                                  
2021-01-04        1511  2021-01-04  92.699997  92.910004  89.220001   
2021-01-05        1512  2021-01-05  90.320000  90.750000  89.099998   
2021-01-06        1513  2021-01-06  89.059998  89.900002  87.889999   
2021-01-07        1514  2021-01-07  88.860001  90.750000  88.269997   
2021-01-08        1515  2021-01-08  90.070000  91.370003  89.529999   
...                ...         ...        ...        ...        ...   
2022-12-23        2009  2022-12-23  75.699997  76.070000  75.500000   
2022-12-27        2010  2022-12-27  75.900002  76.050003  75.540001   
2022-12-28        2011  2022-12-28  75.639999  76.480003  75.500000   
2022-12-29        2012  2022-12-29  76.040001  76.870003  76.029999   
2022-12-30        2013  2022-12-30  76.500000  76.660004  76.220001   


In [55]:
import pickle
pickle.dump(filtered_pairs, open('results_df.pickle', 'wb'))
pickle.dump(backtest_results_raw, open('results_raw.pickle', 'wb'))

In [1]:
import pickle

In [2]:
test = pickle.load(open('results_df.pickle', 'rb'))

In [3]:
kmeans = test['KMeans']

In [10]:
kmeans[(kmeans['SharpeRatio'] > 1.5) & (kmeans['SharpeRatio'] < 10)].to_csv("kmeans_usmarket.csv")

In [5]:
dbscan = test['DBSCAN']

In [6]:
len(dbscan)

166

In [11]:
dbscan[(dbscan['SharpeRatio'] > 1.5) & (dbscan['SharpeRatio'] < 10)].to_csv("dbscan_usmarket.csv")

In [8]:
agglo = test['Agglo']

In [12]:
agglo[(agglo['SharpeRatio'] > 1.5) & (agglo['SharpeRatio'] < 10)].to_csv("agglo_usmarket.csv")

In [ ]:
# def read_stock_price(file_path: str):
#     df = pd.read_csv(file_path, header=[0, 1]).drop(0)
#     # Skipping files with null values for now
#     if df.isnull().values.any():
#         print(f'{file_path} has null values')
#         return None
#     dates = pd.to_datetime(df.iloc[:, 0], utc=True).dt.date
#     dates = pd.to_datetime(dates)
#     dates.name = "Date"
#     df.index = dates
#     return df

In [ ]:
# prices = {}
# files = get_files(path="Data", extension="csv")
# for file_name, file_path in files.items():
#     df = read_stock_price(file_path)
#     if df is not None:
#         prices[file_name] = df

In [ ]:
# results = {}
# for file, stock_price in prices.items():
#     print(f"Getting results for {file}")
#     testing_prices = stock_price.loc[stock_price.index < TESTING_CUTOFF]
#     testing_prices = testing_prices.iloc[:, testing_prices.columns.get_level_values(1) == "Adj Close"]
    
#     stocks = testing_prices.columns.get_level_values(0)
#     testing_prices.columns = stocks
#     pairs = pd.Series([(stocks[i], stocks[j]) for i in range(len(stocks)) for j in range(i+1, len(stocks))])
#     results[file] = find_suitable_pairs(testing_prices, pairs)

In [ ]:
# backtest_res = {}
# for file, (all_pairs_df, filtered_pairs_df) in results.items():
#     for pair in filtered_pairs_df['pair']:
#         print(f"Doing backtesting for: {pair}")
#         stock_prices = prices[file].loc[stock_price.index > TESTING_CUTOFF]
#         res = run_backtest(pair[0], pair[1], stock_prices[pair[0]], stock_prices[pair[1]], PARAMS)
#         backtest_res[file] = res
#         res.analyzers.sharpe.print()
#         res.analyzers.returns.print()
#         print('Ending test\n')
#         print('\n')
        

In [16]:
filtered_pairs

{'KMeans':                                                    pair        pvalue  \
 0        (comms-comms - AMX.csv, comms-comms - BCE.csv)  8.992544e-26   
 1       (comms-comms - AMX.csv, comms-comms - CHTR.csv)  1.607755e-23   
 2      (comms-comms - AMX.csv, comms-comms - CMCSA.csv)  1.447099e-19   
 3        (comms-comms - AMX.csv, comms-comms - DIS.csv)  4.989754e-25   
 4       (comms-comms - AMX.csv, comms-comms - RELX.csv)  2.478912e-24   
 ...                                                 ...           ...   
 4602  (realestate-realestate - ARE.csv, utilities-ut...  2.859040e-29   
 4606  (realestate-realestate - ARE.csv, utilities-ut...  4.190126e-15   
 4608  (realestate-realestate - ARE.csv, utilities-ut...  1.555391e-29   
 4609  (realestate-realestate - ARE.csv, utilities-ut...  0.000000e+00   
 4613  (realestate-realestate - ARE.csv, utilities-ut...  2.256789e-29   
 
       hurst_exp   half_life  avg_cross_count  SharpeRatio  \
 0     -0.000929   30.519410       128